### Introduction
This notebook is the pre-requisite for observability infrastructure, which programmatically deploy an AWS CloudFormation stack using the Python boto3 library. The CloudFormation stack being deployed is defined in the `pre-requisite.yaml` file, which contains the necessary resources and configurations.

Before you execute the notebook, please make sure your Sagemaker Role (if you are running the notebook in Sagemaker) is having permission to:

- IAM Full Access
- Lambda Full Access
- Kinesis Firehose Full Acess
- CreateStack, DescribeStack, DeleteStack for CloudFormation
- CreateCrawler, CreateDataBase, GetCrawler, StopCrawler, DeleteDatabase, DeleteCrawler for Glue
- PutRetentionPolicy, DeleteLogGroup for CloudWatch Logs

After executing the notebook, please copy the last print statement output for `output_dict_select` and paste the results to your `config.py` file.

In [ ]:
%pip install --upgrade pip
%pip install -r requirements.txt

In [ ]:
%pip install ruamel.yaml

In [ ]:
import boto3
import datetime

from ruamel.yaml import YAML
from io import StringIO

In [ ]:
yaml = YAML()
with open('pre-requisite.yaml', 'r') as file:
    yaml_data = yaml.load(file)

In [ ]:
cloudformation_client = boto3.client('cloudformation')
stack_name = 'observability-stack'

In [ ]:
stream = StringIO()
yaml.dump(yaml_data, stream)

template_body = stream.getvalue()

In [ ]:
current_time = datetime.datetime.now()
date_string = current_time.strftime("%Y%m%d%H%M%S")

In [ ]:
response = cloudformation_client.create_stack(
    StackName=stack_name,
    TemplateBody=template_body,
    Parameters=[
        {
            'ParameterKey': 'DateString',
            'ParameterValue': date_string
        }
    ],
    Capabilities=['CAPABILITY_IAM', 'CAPABILITY_NAMED_IAM']
)

In [ ]:
cloudformation_client.get_waiter('stack_create_complete').wait(StackName=stack_name)

In [ ]:
response = cloudformation_client.describe_stacks(StackName=stack_name)
output_dict = {}

for output in response['Stacks'][0]['Outputs']:
    output_dict[output['OutputKey']] = output['OutputValue']

In [ ]:
output_dict_select = {"FIREHOSE_NAME": output_dict["FirehoseDeliveryStreamName"], 
                      "CRAWLER_NAME": output_dict["GlueCrawlerName"]}

In [ ]:
session = boto3.Session()
region = session.region_name

In [ ]:
output_dict_select['REGION'] = region

In [ ]:
print(output_dict_select)

#### Please copy the statement above for `output_dict_select` and paste the results to your `config.py` file